In [ ]:
! nvidia-smi -L

GPU 0: A100-SXM4-40GB (UUID: GPU-3d203e26-4a3e-3592-e586-3080f1d5b7dc)


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from keras.optimizers import RMSprop, SGD
import datetime

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.style.use('ggplot')


In [ ]:
# Load Dataset - Data Augmentation
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

# Randomly Rotate/Flip Image To Prevent Overfitting
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=15,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
    #zoom_range=0.3
    )
datagen.fit(x_train)

# Reshape Data and Convert To Floating Point
img_rows, img_cols = 32, 32
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 3)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 3)
input_shape = (img_rows, img_cols, 3)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalise Data
x_test = (x_test - np.mean(x_train))/np.std(x_train)
x_train = (x_train - np.mean(x_train))/np.std(x_train)

# One-Hot Encoding Of Output Categories
num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

170498071/170498071 [==============================] - 15s 0us/step


#### LeNet-5 Implementation For CIFAR-10
ReLU Timing: 2 min 23.19s

ReLU Accuracy: 0.6398

ELU Timing: 2 min 23.56s

ELU Accuracy: 0.6556




In [ ]:
class LeNet:
  def __init__(self, inputShape=(32,32,3), activationFunction='elu'):
    self.inputShape = inputShape
    self.activationFunction = activationFunction

  def buildModel(self):
    model = tf.keras.Sequential()

    model.add(tf.keras.layers.Conv2D(6, kernel_size=(5,5), strides=(1,1), padding='same', input_shape=input_shape, use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros'))

    model.add(tf.keras.layers.Activation(self.activationFunction))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    model.add(tf.keras.layers.Conv2D(16, kernel_size=(5,5), padding='valid'))
    model.add(tf.keras.layers.Activation(self.activationFunction))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    model.add(tf.keras.layers.Conv2D(120, kernel_size=(5,5), padding='valid'))
    model.add(tf.keras.layers.Activation(self.activationFunction))

    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(84))

    model.add(tf.keras.layers.Dense(num_classes))

    model.add(tf.keras.layers.Activation('softmax'))
    return model

LeNetInstance = LeNet(activationFunction = 'relu')
LeNetReluModel = LeNetInstance.buildModel()
LeNetReluModel.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

LeNetInstance = LeNet(activationFunction = 'elu')
LeNetEluModel = LeNetInstance.buildModel()
LeNetEluModel.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

batch_size = 64
num_classes = 10
epochs = 30

startTime = datetime.datetime.now()
print("Training Of LeNet Using ReLU Activation Function")
LeNetReluModelLog = LeNetReluModel.fit(x_train, y_train,
                      batch_size=batch_size,
                      epochs=epochs,
                      validation_split = 0.1,
                     )
print('Training Time For LeNet Using ReLU: %s' % (datetime.datetime.now() - startTime))

startTime = datetime.datetime.now()
print("Training Of LeNet Using ELU Activation Function")
LeNetEluModelLog = LeNetEluModel.fit(x_train, y_train,
                      batch_size=batch_size,
                      epochs=epochs,
                      validation_split = 0.1,
                     )
print('Training Time For LeNet Using ELU: %s' % (datetime.datetime.now() - startTime))

Training Of LeNet Using ReLU Activation Function
Epoch 1/30
704/704 [==============================] - 14s 6ms/step - loss: 1.5086 - accuracy: 0.4547 - val_loss: 1.2888 - val_accuracy: 0.5398
Epoch 2/30
704/704 [==============================] - 4s 5ms/step - loss: 1.2252 - accuracy: 0.5646 - val_loss: 1.2382 - val_accuracy: 0.5556
Epoch 3/30
704/704 [==============================] - 3s 5ms/step - loss: 1.1029 - accuracy: 0.6123 - val_loss: 1.1754 - val_accuracy: 0.5918
Epoch 4/30
704/704 [==============================] - 3s 5ms/step - loss: 1.0220 - accuracy: 0.6389 - val_loss: 1.0421 - val_accuracy: 0.6414
Epoch 5/30
704/704 [==============================] - 3s 5ms/step - loss: 0.9546 - accuracy: 0.6636 - val_loss: 1.0854 - val_accuracy: 0.6242
Epoch 6/30
704/704 [==============================] - 3s 5ms/step - loss: 0.8994 - accuracy: 0.6850 - val_loss: 1.0140 - val_accuracy: 0.6524
Epoch 7/30
704/704 [==============================] - 3s 5ms/step - loss: 0.8622 - accuracy: 0.696

#### ResNet-50 Implementation For CIFAR-10 (Using Colab Pro)
ReLU Timing: 36 min 43.61s

ReLU Accuracy: 0.8298

ELU Timing: 37 min 32.61s

ELU Accuracy: 0.8176

In [ ]:
class ResNet50:
  def __init__(self, activationFunction='relu'):
    self.activationFunction = activationFunction

  def convolutionalBlock(self, X, f, filters, s = 2):
    F1, F2, F3 = filters
    X_shortcut = X
    X = tf.keras.layers.Conv2D(F1, (1, 1), strides = (s,s), kernel_initializer = "glorot_uniform")(X)
    X = tf.keras.layers.BatchNormalization(axis = 3)(X)
    X = tf.keras.layers.Activation(self.activationFunction)(X)
    X = tf.keras.layers.Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', kernel_initializer = "glorot_uniform")(X)
    X = tf.keras.layers.BatchNormalization(axis = 3)(X)
    X = tf.keras.layers.Activation(self.activationFunction)(X)
    X = tf.keras.layers.Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', kernel_initializer = "glorot_uniform")(X)
    X = tf.keras.layers.BatchNormalization(axis = 3)(X)
    X_shortcut = tf.keras.layers.Conv2D(filters = F3, kernel_size = (1, 1), strides = (s,s), padding = 'valid', kernel_initializer = "glorot_uniform")(X_shortcut)
    X_shortcut = tf.keras.layers.BatchNormalization(axis = 3)(X_shortcut)
    X = tf.keras.layers.Add()([X_shortcut, X])
    X = tf.keras.layers.Activation(self.activationFunction)(X)
    return X

  def identityBlock(self, X, f, filters):
    F1, F2, F3 = filters
    X_shortcut = X
    X = tf.keras.layers.Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', kernel_initializer = "glorot_uniform")(X)
    X = tf.keras.layers.BatchNormalization(axis = 3)(X)
    X = tf.keras.layers.Activation(self.activationFunction)(X)
    X = tf.keras.layers.Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', kernel_initializer = "glorot_uniform")(X)
    X = tf.keras.layers.BatchNormalization(axis = 3)(X)
    X = tf.keras.layers.Activation(self.activationFunction)(X)
    X = tf.keras.layers.Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', kernel_initializer = "glorot_uniform")(X)
    X = tf.keras.layers.BatchNormalization(axis = 3)(X)
    X = tf.keras.layers.Add()([X_shortcut, X])
    X = tf.keras.layers.Activation(self.activationFunction)(X)
    return X

  def buildModel(self, classes=10):
    X_input = tf.keras.layers.Input(input_shape)
    X = tf.keras.layers.Resizing(224, 224)(X_input)
    X = tf.keras.layers.ZeroPadding2D((3, 3))(X)
    X = tf.keras.layers.Conv2D(64, (7, 7), strides = (2, 2), kernel_initializer = "glorot_uniform")(X)
    X = tf.keras.layers.BatchNormalization(axis = 3)(X)
    X = tf.keras.layers.Activation(self.activationFunction)(X)
    X = tf.keras.layers.MaxPooling2D((3, 3), strides=(2, 2))(X)
    X = self.convolutionalBlock(X, f = 3, filters = [64, 64, 256], s = 1)
    X = self.identityBlock(X, 3, [64, 64, 256])
    X = self.identityBlock(X, 3, [64, 64, 256])
    X = self.convolutionalBlock(X, f = 3, filters = [128, 128, 512], s = 2)
    X = self.identityBlock(X, 3, [128, 128, 512])
    X = self.identityBlock(X, 3, [128, 128, 512])
    X = self.identityBlock(X, 3, [128, 128, 512])
    X = self.convolutionalBlock(X, f = 3, filters = [256, 256, 1024], s = 2)
    X = self.identityBlock(X, 3, [256, 256, 1024])
    X = self.identityBlock(X, 3, [256, 256, 1024])
    X = self.identityBlock(X, 3, [256, 256, 1024])
    X = self.identityBlock(X, 3, [256, 256, 1024])
    X = self.identityBlock(X, 3, [256, 256, 1024])
    X = self.convolutionalBlock(X, f = 3, filters = [512, 512, 2048], s = 2)
    X = self.identityBlock(X, 3, [512, 512, 2048])
    X = self.identityBlock(X, 3, [512, 512, 2048])
    X = tf.keras.layers.AveragePooling2D(pool_size=(2, 2))(X)
    X = tf.keras.layers.Flatten()(X)
    X = tf.keras.layers.Dense(classes, activation='softmax', kernel_initializer = "glorot_uniform")(X)
    model = tf.keras.Model(inputs = X_input, outputs = X)
    return model

ResInstance = ResNet50(activationFunction = 'relu')
ResnetReluModel = ResInstance.buildModel()
ResnetReluModel.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

ResInstance = ResNet50(activationFunction = 'elu')
ResnetEluModel = ResInstance.buildModel()
ResnetEluModel.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

batch_size = 64
num_classes = 10
epochs = 30

startTime = datetime.datetime.now()
print("Training Of ResNet-50 Using ReLU Activation Function")
model_log = ResnetReluModel.fit(x_train, y_train,
                      batch_size=batch_size,
                      epochs=epochs,
                      validation_split = 0.1,
                     )
print('Training Time For ResNet-50 Using ReLU: %s' % (datetime.datetime.now() - startTime))

startTime = datetime.datetime.now()
print("Training Of ResNet-50 Using ELU Activation Function")
model_log = ResnetEluModel.fit(x_train, y_train,
                      batch_size=batch_size,
                      epochs=epochs,
                      validation_split = 0.1,
                     )

print('Training Time For ResNet-50 Using ELU: %s' % (datetime.datetime.now() - startTime))

Training Of ResNet-50 Using ReLU Activation Function
Epoch 1/30
704/704 [==============================] - 81s 106ms/step - loss: 1.8039 - accuracy: 0.4259 - val_loss: 1.8303 - val_accuracy: 0.4422
Epoch 2/30
704/704 [==============================] - 73s 104ms/step - loss: 1.0960 - accuracy: 0.6136 - val_loss: 0.9422 - val_accuracy: 0.6654
Epoch 3/30
704/704 [==============================] - 73s 104ms/step - loss: 0.8331 - accuracy: 0.7085 - val_loss: 0.8848 - val_accuracy: 0.7062
Epoch 4/30
704/704 [==============================] - 73s 104ms/step - loss: 0.6544 - accuracy: 0.7722 - val_loss: 0.6991 - val_accuracy: 0.7618
Epoch 5/30
704/704 [==============================] - 73s 104ms/step - loss: 0.5307 - accuracy: 0.8155 - val_loss: 0.8091 - val_accuracy: 0.7406
Epoch 6/30
704/704 [==============================] - 73s 103ms/step - loss: 0.4398 - accuracy: 0.8462 - val_loss: 0.6377 - val_accuracy: 0.7834
Epoch 7/30
704/704 [==============================] - 73s 103ms/step - loss: 

#### ResNet-34 For CIFAR-10 (Using Colab Pro)
ReLU Timing: 22 min 52.96s

ReLU Accuracy: 0.8270

ELU Timing: 23 min 01.17s

ELU Accuracy: 0.8232

In [ ]:
class ResNetImproved:
  def __init__(self, activationFunction='relu'):
    self.activationFunction = activationFunction

  def convolutionalBlock(self, X, f, filters, s = 2):
    X_shortcut = X
    X = tf.keras.layers.Conv2D(filters, (f, f), strides = (s,s), kernel_initializer = "glorot_uniform",)(X)
    X = tf.keras.layers.BatchNormalization(axis = 3)(X)
    X = tf.keras.layers.Activation(self.activationFunction)(X)
    X = tf.keras.layers.Conv2D(filters, (f, f), strides = (1,1), padding = 'same', kernel_initializer = "glorot_uniform",)(X)
    X = tf.keras.layers.BatchNormalization(axis = 3)(X)
    X_shortcut = tf.keras.layers.Conv2D(filters, (f, f), strides = (s,s), padding = 'valid', kernel_initializer = "glorot_uniform",)(X_shortcut)
    X_shortcut = tf.keras.layers.BatchNormalization(axis = 3)(X_shortcut)
    X = tf.keras.layers.Add()([X_shortcut, X])
    X = tf.keras.layers.Activation(self.activationFunction)(X)
    return X

  def identityBlock(self, X, f, filters):
    X_shortcut = X
    X = tf.keras.layers.Conv2D(filters, (f, f), strides = (1,1), padding = 'same', kernel_initializer = "glorot_uniform",)(X)
    X = tf.keras.layers.BatchNormalization(axis = 3)(X)
    X = tf.keras.layers.Activation(self.activationFunction)(X)
    X = tf.keras.layers.Conv2D(filters, (f, f), strides = (1,1), padding = 'same', kernel_initializer = "glorot_uniform",)(X)
    X = tf.keras.layers.BatchNormalization(axis = 3)(X)
    X = tf.keras.layers.Add()([X_shortcut, X])
    X = tf.keras.layers.Activation(self.activationFunction)(X)
    return X

  def buildModel(self, classes=10):
    X_input = tf.keras.layers.Input(input_shape)
    X = tf.keras.layers.Resizing(224, 224)(X_input)
    X = tf.keras.layers.ZeroPadding2D((3, 3))(X)
    X = tf.keras.layers.Conv2D(64, (7, 7), strides = (2, 2), kernel_initializer = "glorot_uniform",
                               kernel_regularizer=tf.keras.regularizers.L1L2(l1=1e-5, l2=1e-4),
                                         bias_regularizer=tf.keras.regularizers.L2(1e-4),
                                         activity_regularizer=tf.keras.regularizers.L2(1e-5))(X)
    X = tf.keras.layers.BatchNormalization(axis = 3)(X)
    X = tf.keras.layers.Activation(self.activationFunction)(X)
    X = tf.keras.layers.MaxPooling2D((3, 3), strides=(2, 2))(X)
    X = self.convolutionalBlock(X, f = 3, filters = 64, s = 1)
    X = self.identityBlock(X, 3, 64)
    X = self.identityBlock(X, 3, 64)
    X = self.convolutionalBlock(X, f = 3, filters = 128, s = 2)
    X = self.identityBlock(X, 3, 128)
    X = self.identityBlock(X, 3, 128)
    X = self.identityBlock(X, 3, 128)
    X = self.convolutionalBlock(X, f = 3, filters = 256, s = 2)
    X = self.identityBlock(X, 3, 256)
    X = self.identityBlock(X, 3, 256)
    X = self.identityBlock(X, 3, 256)
    X = self.identityBlock(X, 3, 256)
    X = self.identityBlock(X, 3, 256)
    X = self.convolutionalBlock(X, f = 3, filters = 512, s = 2)
    X = self.identityBlock(X, 3, 512)
    X = self.identityBlock(X, 3, 512)
    X = tf.keras.layers.AveragePooling2D(pool_size=(2, 2))(X)
    X = tf.keras.layers.Flatten()(X)
    X = tf.keras.layers.Dense(classes, activation='softmax', kernel_initializer = "glorot_uniform",
                              kernel_regularizer=tf.keras.regularizers.L1L2(l1=1e-5, l2=1e-4),
                                         bias_regularizer=tf.keras.regularizers.L2(1e-4),
                                         activity_regularizer=tf.keras.regularizers.L2(1e-5))(X)
    model = tf.keras.Model(inputs = X_input, outputs = X)
    return model


ResInstance = ResNetImproved(activationFunction = 'relu')
ResnetReluModel = ResInstance.buildModel()
ResnetReluModel.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.01, clipnorm = 1, decay=0.005), metrics=['accuracy'])

ResInstance = ResNetImproved(activationFunction = 'elu')
ResnetEluModel = ResInstance.buildModel()
ResnetEluModel.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.01, clipnorm = 1, decay=0.005), metrics=['accuracy'])

batch_size = 64
num_classes = 10
epochs = 30

startTime = datetime.datetime.now()
print("Training Of ResNet-34 Using ReLU Activation Function")
model_log = ResnetReluModel.fit(x_train, y_train,
                      batch_size=batch_size,
                      epochs=epochs,
                      validation_split = 0.1,
                     )
print('Training Time For ResNet-34 Using ReLU: %s' % (datetime.datetime.now() - startTime))

startTime = datetime.datetime.now()
print("Training Of ResNet-34 Using ELU Activation Function")
model_log = ResnetEluModel.fit(x_train, y_train,
                      batch_size=batch_size,
                      epochs=epochs,
                      validation_split = 0.1,
                     )

print('Training Time For ResNet-34 Using ELU: %s' % (datetime.datetime.now() - startTime))

Training Of ResNet-34 Using ReLU Activation Function
Epoch 1/30
704/704 [==============================] - 54s 65ms/step - loss: 2.6083 - accuracy: 0.3031 - val_loss: 1.8964 - val_accuracy: 0.3858
Epoch 2/30
704/704 [==============================] - 45s 64ms/step - loss: 1.7075 - accuracy: 0.4478 - val_loss: 1.5910 - val_accuracy: 0.4870
Epoch 3/30
704/704 [==============================] - 45s 64ms/step - loss: 1.4052 - accuracy: 0.5440 - val_loss: 1.3202 - val_accuracy: 0.5662
Epoch 4/30
704/704 [==============================] - 45s 64ms/step - loss: 1.1750 - accuracy: 0.6238 - val_loss: 1.3600 - val_accuracy: 0.5782
Epoch 5/30
704/704 [==============================] - 45s 64ms/step - loss: 0.9876 - accuracy: 0.6870 - val_loss: 0.9872 - val_accuracy: 0.6834
Epoch 6/30
704/704 [==============================] - 44s 63ms/step - loss: 0.8155 - accuracy: 0.7450 - val_loss: 0.8947 - val_accuracy: 0.7066
Epoch 7/30
704/704 [==============================] - 45s 64ms/step - loss: 0.6786 

#### ResNet 18 (Best Model) (No Colab Pro)

ReLU Timing: 1h 21min 27.45s

ReLU Timing (Adjusted To Colab Pro): 22 min 23.95s

*(A100 is 1.47x faster than V100, which is 2.46x faster than T4)*

ReLU Accuracy: 0.8310


In [ ]:
class ResNetImproved:
  def __init__(self, activationFunction='relu'):
    self.activationFunction = activationFunction

  def convolutionalBlock(self, X, f, filters, s = 2):
    X_shortcut = X
    X = tf.keras.layers.Conv2D(filters, (f, f), strides = (s,s), kernel_initializer = "glorot_uniform",)(X)
    X = tf.keras.layers.BatchNormalization(axis = 3)(X)
    X = tf.keras.layers.Activation(self.activationFunction)(X)
    X = tf.keras.layers.Conv2D(filters, (f, f), strides = (1,1), padding = 'same', kernel_initializer = "glorot_uniform",)(X)
    X = tf.keras.layers.BatchNormalization(axis = 3)(X)
    X_shortcut = tf.keras.layers.Conv2D(filters, (f, f), strides = (s,s), padding = 'valid', kernel_initializer = "glorot_uniform",)(X_shortcut)
    X_shortcut = tf.keras.layers.BatchNormalization(axis = 3)(X_shortcut)
    X = tf.keras.layers.Add()([X_shortcut, X])
    X = tf.keras.layers.Activation(self.activationFunction)(X)
    return X

  def identityBlock(self, X, f, filters):
    X_shortcut = X
    X = tf.keras.layers.Conv2D(filters, (f, f), strides = (1,1), padding = 'same', kernel_initializer = "glorot_uniform",)(X)
    X = tf.keras.layers.BatchNormalization(axis = 3)(X)
    X = tf.keras.layers.Activation(self.activationFunction)(X)
    X = tf.keras.layers.Conv2D(filters, (f, f), strides = (1,1), padding = 'same', kernel_initializer = "glorot_uniform",)(X)
    X = tf.keras.layers.BatchNormalization(axis = 3)(X)
    X = tf.keras.layers.Add()([X_shortcut, X])
    X = tf.keras.layers.Activation(self.activationFunction)(X)
    return X

  def buildModel(self, classes=10):
    X_input = tf.keras.layers.Input(input_shape)
    X = tf.keras.layers.Resizing(224, 224)(X_input)
    X = tf.keras.layers.ZeroPadding2D((3, 3))(X)
    X = tf.keras.layers.Conv2D(64, (7, 7), strides = (2, 2), kernel_initializer = "glorot_uniform",
                               kernel_regularizer=tf.keras.regularizers.L1L2(l1=1e-5, l2=1e-4),
                                         bias_regularizer=tf.keras.regularizers.L2(1e-4),
                                         activity_regularizer=tf.keras.regularizers.L2(1e-5))(X)
    X = tf.keras.layers.BatchNormalization(axis = 3)(X)
    X = tf.keras.layers.Activation(self.activationFunction)(X)
    X = tf.keras.layers.MaxPooling2D((3, 3), strides=(2, 2))(X)
    X = self.convolutionalBlock(X, f = 3, filters = 64, s = 1)
    X = self.identityBlock(X, 3, 64)
    X = self.convolutionalBlock(X, f = 3, filters = 128, s = 2)
    X = self.identityBlock(X, 3, 128)
    X = self.convolutionalBlock(X, f = 3, filters = 256, s = 2)
    X = self.identityBlock(X, 3, 256)
    X = self.convolutionalBlock(X, f = 3, filters = 512, s = 2)
    X = self.identityBlock(X, 3, 512)
    X = tf.keras.layers.AveragePooling2D(pool_size=(2, 2))(X)
    X = tf.keras.layers.Flatten()(X)
    X = tf.keras.layers.Dense(classes, activation='softmax', kernel_initializer = "glorot_uniform",
                              kernel_regularizer=tf.keras.regularizers.L1L2(l1=1e-5, l2=1e-4),
                                         bias_regularizer=tf.keras.regularizers.L2(1e-4),
                                         activity_regularizer=tf.keras.regularizers.L2(1e-5))(X)
    model = tf.keras.Model(inputs = X_input, outputs = X)
    return model


ResInstance = ResNetImproved(activationFunction = 'relu')
ResnetReluModel = ResInstance.buildModel()
ResnetReluModel.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.01, clipnorm = 1, decay=0.005), metrics=['accuracy'])

ResInstance = ResNetImproved(activationFunction = 'elu')
ResnetEluModel = ResInstance.buildModel()
ResnetEluModel.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.01, clipnorm = 1, decay=0.005), metrics=['accuracy'])

batch_size = 64
num_classes = 10
epochs = 30

startTime = datetime.datetime.now()
print("Training Of ResNet-18 Using ReLU Activation Function")
model_log = ResnetReluModel.fit(x_train, y_train,
                      batch_size=batch_size,
                      epochs=epochs,
                      validation_split = 0.1,
                     )
print('Training Time For ResNet-18 Using ReLU: %s' % (datetime.datetime.now() - startTime))

startTime = datetime.datetime.now()
print("Training Of ResNet-18 Using ELU Activation Function")
model_log = ResnetEluModel.fit(x_train, y_train,
                      batch_size=batch_size,
                      epochs=epochs,
                      validation_split = 0.1,
                     )

print('Training Time For ResNet-18 Using ELU: %s' % (datetime.datetime.now() - startTime))

Training Of ResNet-18 Using ReLU Activation Function
Epoch 1/30
704/704 [==============================] - 176s 231ms/step - loss: 2.3503 - accuracy: 0.3366 - val_loss: 1.8313 - val_accuracy: 0.4364
Epoch 2/30
704/704 [==============================] - 161s 229ms/step - loss: 1.5643 - accuracy: 0.5030 - val_loss: 1.4340 - val_accuracy: 0.5600
Epoch 3/30
704/704 [==============================] - 161s 229ms/step - loss: 1.2164 - accuracy: 0.6212 - val_loss: 1.0940 - val_accuracy: 0.6554
Epoch 4/30
704/704 [==============================] - 162s 229ms/step - loss: 0.9496 - accuracy: 0.7069 - val_loss: 0.8690 - val_accuracy: 0.7364
Epoch 5/30
704/704 [==============================] - 161s 229ms/step - loss: 0.7457 - accuracy: 0.7746 - val_loss: 0.7912 - val_accuracy: 0.7562
Epoch 6/30
704/704 [==============================] - 161s 229ms/step - loss: 0.5910 - accuracy: 0.8258 - val_loss: 0.6861 - val_accuracy: 0.7854
Epoch 7/30
704/704 [==============================] - 162s 230ms/step -

#### VGG-16 Implementation For CIFAR-10 (Using Colab Pro)
ReLU Timing: 1 h 1 min 28.50s

ReLU Accuracy: 0.8524

ELU Timing: 1 h 02 min 50.60s

ELU Accuracy: 0.8636

In [ ]:
class VGG16:

  def __init__(self, inputShape=(32,32,3), weightDecay = 0.0005, activationFunction='elu'):
    self.inputShape = inputShape
    self.weightDecay = weightDecay
    self.activationFunction = activationFunction

  def buildModel(self):
    model = tf.keras.Sequential()

    model.add(tf.keras.layers.Input(shape=input_shape))
    model.add(tf.keras.layers.Resizing(224, 224))

    # Input Block
    model.add(tf.keras.layers.Conv2D(64, (3, 3), padding='same', kernel_regularizer=tf.keras.regularizers.L2(self.weightDecay)))
    model.add(tf.keras.layers.Activation(self.activationFunction))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(0.3))

    self.ConvolutionBlock(model, 64)
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

    self.ConvolutionBlock(model, 128, 0.4)
    self.ConvolutionBlock(model, 128)
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

    self.ConvolutionBlock(model, 256, 0.4)
    self.ConvolutionBlock(model, 256, 0.4)
    self.ConvolutionBlock(model, 256)
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

    self.ConvolutionBlock(model, 512, 0.4)
    self.ConvolutionBlock(model, 512, 0.4)
    self.ConvolutionBlock(model, 512)
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

    self.ConvolutionBlock(model, 512, 0.4)
    self.ConvolutionBlock(model, 512, 0.4)
    self.ConvolutionBlock(model, 512)
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(512,kernel_regularizer=tf.keras.regularizers.L2(self.weightDecay)))
    model.add(tf.keras.layers.Activation(self.activationFunction))
    model.add(tf.keras.layers.BatchNormalization())

    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(10))
    model.add(tf.keras.layers.Activation('softmax'))

    return model

  def ConvolutionBlock(self, model, filter, dropout=0):
    model.add(tf.keras.layers.Conv2D(filter, (3, 3), padding='same',kernel_regularizer=tf.keras.regularizers.L2(self.weightDecay)))
    model.add(tf.keras.layers.Activation(self.activationFunction))
    model.add(tf.keras.layers.BatchNormalization())

    if dropout != 0:
      model.add(tf.keras.layers.Dropout(dropout))

VGGInstance = VGG16(activationFunction = 'relu')
VGGReluModel = VGGInstance.buildModel()
VGGReluModel.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.01, clipnorm = 1, decay=0.005), metrics=['accuracy'])

VGGInstance = VGG16(activationFunction = 'elu')
VGGEluModel = VGGInstance.buildModel()
VGGEluModel.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.01, clipnorm = 1, decay=0.005), metrics=['accuracy'])

batch_size = 64
num_classes = 10
epochs = 30

startTime = datetime.datetime.now()
print("Training Of VGG-16 Using ReLU Activation Function")
VGGReluModelLog = VGGReluModel.fit(x_train, y_train,
                      batch_size=batch_size,
                      epochs=epochs,
                      validation_split = 0.1,
                     )
print('Training Time For VGG-16 Using ReLU: %s' % (datetime.datetime.now() - startTime))

startTime = datetime.datetime.now()
print("Training Of VGG-16 Using ELU Activation Function")
VGGEluModelLog = VGGEluModel.fit(x_train, y_train,
                      batch_size=batch_size,
                      epochs=epochs,
                      validation_split = 0.1,
                     )
print('Training Time For VGG-16 Using ELU: %s' % (datetime.datetime.now() - startTime))

Training Of VGG-16 Using ReLU Activation Function
Epoch 1/30
704/704 [==============================] - 127s 175ms/step - loss: 5.2095 - accuracy: 0.2705 - val_loss: 3.9431 - val_accuracy: 0.1058
Epoch 2/30
704/704 [==============================] - 123s 174ms/step - loss: 2.7521 - accuracy: 0.3420 - val_loss: 3.3658 - val_accuracy: 0.1572
Epoch 3/30
704/704 [==============================] - 123s 174ms/step - loss: 2.4313 - accuracy: 0.3721 - val_loss: 3.2042 - val_accuracy: 0.1524
Epoch 4/30
704/704 [==============================] - 123s 174ms/step - loss: 2.2963 - accuracy: 0.4114 - val_loss: 2.2444 - val_accuracy: 0.4568
Epoch 5/30
704/704 [==============================] - 123s 174ms/step - loss: 2.1315 - accuracy: 0.5547 - val_loss: 2.2351 - val_accuracy: 0.5528
Epoch 6/30
704/704 [==============================] - 123s 174ms/step - loss: 1.9370 - accuracy: 0.6469 - val_loss: 1.9402 - val_accuracy: 0.6494
Epoch 7/30
704/704 [==============================] - 123s 174ms/step - lo

#### AlexNet Implementation For CIFAR-10 (Using Colab Pro)
ReLU Timing: 32 min 24.64s

ReLU Accuracy: 0.8450

ELU Timing: 32 min 15.08s

ELU Accuracy: 0.8336

In [ ]:
class AlexNet:

  def __init__(self, inputShape=(32,32,3), activationFunction='relu'):
    self.inputShape = inputShape
    self.activationFunction = activationFunction

  def buildModel(self):
    model = tf.keras.Sequential()

    model.add(tf.keras.layers.Resizing(224, 224))

    # Convolutional Layer 1
    model.add(tf.keras.layers.Conv2D(filters=96, input_shape=self.inputShape, kernel_size=(11,11), strides=(4,4), padding='same'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation(self.activationFunction))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

    # Convolutional Layer 2
    model.add(tf.keras.layers.Conv2D(filters=256, kernel_size=(5, 5), strides=(1,1), padding='same'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation(self.activationFunction))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

    # Convolutional Layer 3
    model.add(tf.keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation(self.activationFunction))

    # Convolutional Layer 4
    model.add(tf.keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation(self.activationFunction))

    # Convolutional Layer 5
    model.add(tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation(self.activationFunction))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

    #Pass To Fully Connected layer
    model.add(tf.keras.layers.Flatten())

    # Fully Connected Layer 1
    model.add(tf.keras.layers.Dense(4096, input_shape=self.inputShape))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation(self.activationFunction))
    model.add(tf.keras.layers.Dropout(0.4))

    # Fully Connected Layer 2
    model.add(tf.keras.layers.Dense(4096))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation(self.activationFunction))
    model.add(tf.keras.layers.Dropout(0.4))

    # Fully Connected Layer 3
    model.add(tf.keras.layers.Dense(1000))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation(self.activationFunction))
    model.add(tf.keras.layers.Dropout(0.4))

    #Output Layer
    model.add(tf.keras.layers.Dense(10))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation('softmax'))

    return model

AlexNetInstance = AlexNet(activationFunction = 'relu')
AlexNetReluModel = AlexNetInstance.buildModel()
AlexNetReluModel.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

AlexNetInstance = AlexNet(activationFunction = 'elu')
AlexNetEluModel = AlexNetInstance.buildModel()
AlexNetEluModel.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

batch_size = 64
num_classes = 10
epochs = 30

startTime = datetime.datetime.now()
print("Training Of AlexNet Using ReLU Activation Function")
AlexNetReluModelLog = AlexNetReluModel.fit(x_train, y_train,
                      batch_size=batch_size,
                      epochs=epochs,
                      validation_split = 0.1,
                     )
print('Training Time For AlexNet Using ReLU: %s' % (datetime.datetime.now() - startTime))

startTime = datetime.datetime.now()
print("Training Of AlexNet Using ELU Activation Function")
AlexNetEluModelLog = AlexNetEluModel.fit(x_train, y_train,
                      batch_size=batch_size,
                      epochs=epochs,
                      validation_split = 0.1,
                     )
print('Training Time For AlexNet Using ELU: %s' % (datetime.datetime.now() - startTime))

Training Of AlexNet Using ReLU Activation Function
Epoch 1/30
704/704 [==============================] - 67s 91ms/step - loss: 1.3691 - accuracy: 0.5264 - val_loss: 1.1720 - val_accuracy: 0.6074
Epoch 2/30
704/704 [==============================] - 64s 91ms/step - loss: 0.9476 - accuracy: 0.6852 - val_loss: 0.9194 - val_accuracy: 0.6934
Epoch 3/30
704/704 [==============================] - 64s 90ms/step - loss: 0.7445 - accuracy: 0.7559 - val_loss: 0.8056 - val_accuracy: 0.7284
Epoch 4/30
704/704 [==============================] - 64s 90ms/step - loss: 0.6108 - accuracy: 0.8017 - val_loss: 0.6738 - val_accuracy: 0.7732
Epoch 5/30
704/704 [==============================] - 64s 91ms/step - loss: 0.4991 - accuracy: 0.8389 - val_loss: 0.6393 - val_accuracy: 0.7852
Epoch 6/30
704/704 [==============================] - 64s 90ms/step - loss: 0.3995 - accuracy: 0.8705 - val_loss: 0.6040 - val_accuracy: 0.8084
Epoch 7/30
704/704 [==============================] - 63s 90ms/step - loss: 0.3103 - 